# Load Model

In [ ]:
import joblib

SM_OUTPUT_DATA_DIR = '../output-model'
model_path = SM_OUTPUT_DATA_DIR + '/best_model.joblib'
model = joblib.load(model_path)

## Get Azure ML Workspace

In [ ]:
import azureml
from azureml.core import Workspace
from azureml.core.authentication import ServicePrincipalAuthentication

workspace = Workspace.from_config()

## Register the model

In [ ]:
from azureml.core.model import Model

model_name = 'fraud-model'
model_description = 'Fraud model'

model_azure = Model.register(model_path = model_path,
                             model_name = model_name,
                             description = model_description,
                             workspace = workspace,
                             tags={'Framework': "SKlearn", 'Type': "Classification", 'Model': "RandomForest"}
                             )

## Deploy

### Entry Script

In [ ]:
%%writefile score.py

import joblib
import json
import pandas as pd
import os
import time
import random
from datetime import datetime

# Called when the deployed service starts
def init():
    global model

    # Get the path where the deployed model can be found.
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), './best_model.joblib')
    
    # Load model
    model = joblib.load(model_path)

# Handle requests to the service
def run(data):
  input_data = json.loads(data)
  input_data = pd.DataFrame([input_data])
  
  # Return the prediction
  prediction = predict(input_data)
  
  #end_time = datetime.now()
  #diff_time = end_time - current_time
  #prediction.update({'elapsed_time': diff_time.microseconds / 1000})
  
  return prediction

def predict(data):
  prediction = model.predict(data)[0]
  return {"prediction": str(prediction)}

### Inference Config

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
env = Environment(name='sklearn_env')

conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("joblib")
conda_dep.add_pip_package("pandas")
conda_dep.add_pip_package("sklearn")

# Adds dependencies to PythonSection of myenv
env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=env)

### AKS

###  Create a new inference cluster (if not exists)

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
aks_name = 'aks-tesserato-01'

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=workspace, name=aks_name)
    print('Found existing cluster, we can use it.')
except ComputeTargetException:
    # Create the cluster
    aks_target = ComputeTarget.create(workspace = workspace,
                                name = aks_name,
                                provisioning_configuration = prov_config)

    # Wait for the create process to complete
    aks_target.wait_for_completion(show_output = True)

## Deploy the model

In [ ]:
from azureml.core.webservice import AksWebservice
from azureml.core.compute import AksCompute

endpoint_name = 'api-fraud-dev'

aks_service = Model.deploy(workspace=workspace,
                           name=endpoint_name,
                           models=[model_azure],
                           inference_config=inference_config,
                           deployment_target=aks_target,                           
                           overwrite=True)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

#### Call the API

In [ ]:
import pandas as pd
import json
import requests

payload = {"type":3.0,
           "amount":1864.28,
           "oldbalanceOrg":21249.0,
           "newbalanceOrig": 19384.72,
           "oldbalanceDest":0.0,
           "newbalanceDest":0.0,
           "hour":1.0,
           "dayOfMonth":1.0,
           "isMerchantDest":1.0,
           "errorBalanceOrig":0.0,
           "errorBalanceDest":1864.28}

# Model prediction
print(f'Model prediction (directly): {model.predict(pd.DataFrame.from_dict([payload]))[0]}') 

# Call the API with the same request
headers = {
  'Content-Type': 'application/json'
}

prod_service_key = aks_service.get_keys()[0] if len(aks_service.get_keys()) > 0 else None

headers["Authorization"] = "Bearer {service_key}".format(service_key=prod_service_key)

response = requests.request("POST", aks_service.scoring_uri, headers=headers, data=json.dumps(payload))
print(f'Model prediction (directly): {response.text}')
